In [ ]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import tensorflow as tf

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [ ]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)


In [ ]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r
def readcsv(path):
    f = open(path,'r')
    f.readline()
    for l in f:
        u,b = l.strip().split(',')
        yield u,b

In [ ]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

pairs_Ratings =[l+ (0,) for l in readcsv("pairs_Rating.csv")]
pairs_Read = [l for l in readcsv("pairs_Read.csv")]

In [ ]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in allRatings:
  ratingsPerUser[u].append((b,r))
  ratingsPerItem[b].append((u,r))

users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)

# Rating Prediction


In [ ]:
b_allRatings = []
userRatings = defaultdict(list)

for user,book,r in allRatings:
  r = int(r)
  b_allRatings.append(r)
  userRatings[user].append(r)

globalAverage = sum(b_allRatings) / len(b_allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])




## Bias Model


In [ ]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

alpha = globalAverage
betaU = defaultdict(float)
betaI = defaultdict(float)
### DOING THIS
ratingsTrain = allRatings
#from hw3 
# def iterate(lamb):
#     newAlpha = 0
#     for u,b,r in ratingsTrain:
#         newAlpha += r - (betaU[u] + betaI[b])
#     alpha = newAlpha / len(ratingsTrain)
#     for u in ratingsPerUser:
#         newBetaU = 0
#         for b,r in ratingsPerUser[u]:
#             newBetaU += r - (alpha + betaI[b])
#         betaU[u] = newBetaU / (lamb + len(ratingsPerUser[u]))
#     for b in ratingsPerItem:
#         newBetaI = 0
#         for u,r in ratingsPerItem[b]:
#             newBetaI += r - (alpha + betaU[u])
#         betaI[b] = newBetaI / (lamb + len(ratingsPerItem[b]))
#     mse = 0
#     for u,b,r in ratingsTrain:
#         prediction = alpha + betaU[u] + betaI[b]
#         mse += (r - prediction)**2
#     regularizer = 0
#     for u in betaU:
#         regularizer += betaU[u]**2
#     for b in betaI:
#         regularizer += betaI[b]**2
#     mse /= len(ratingsTrain)
#     return mse, mse + lamb*regularizer

class lf_model():
  def __init__(self):
    self.alpha = globalAverage
    self.userBiases = defaultdict(float)
    self.itemBiases = defaultdict(float)

  def fit(self,lam):
    itrs = 100
    for i in range(itrs):
      if i %10 == 0:
        print(i)
      for d in ratingsTrain:
        u,i,r = d[0], d[1],d[2]
        self.alpha += (r - (self.userBiases[u] + self.itemBiases[i]))
      self.alpha /= len(ratingsTrain)
      for u in ratingsPerUser:
        items = [b for b, r in ratingsPerUser[u]]
        ratings = [r for b, r in ratingsPerUser[u]]
        self.userBiases[u] = sum([ratings[j] - (self.alpha + self.itemBiases[items[j]]) for j in range(len(items))])/(lam + len(set(items)))

      for i in ratingsPerItem:
        users = [u for u, r in ratingsPerItem[i]]
        ratings = [r for u, r in ratingsPerItem[i]]
        self.itemBiases[i] = sum([ratings[j] - (self.alpha + self.userBiases[users[j]]) for j in range(len(users))])/(lam + len(set(users)))
    return self.alpha, self.userBiases, self.itemBiases
  
  def prediction(self,user, item):
    return self.alpha + self.userBiases[user] + self.itemBiases[item]



In [ ]:
lm = 5
mse,objective = iterate(lm)
newMSE,newObjective = iterate(lm)
iterations = 2
while iterations < 10 or objective - newObjective > 0.0001:
    mse, objective = newMSE, newObjective
    newMSE, newObjective = iterate(lm)
    iterations += 1
    # print("Objective after "
    #     + str(iterations) + " iterations = " + str(newObjective))
    if iterations%10 == 0:
      print("MSE after "
        + str(iterations) + " iterations = " + str(newMSE))
    


MSE after 10 iterations = 1.1569389605313316


In [ ]:
reg = 5
mod = lf_model()
bestalpha, bestuserBiases, bestitemBiases = mod.fit(lam = reg)

0
10
20
30
40
50
60
70
80
90


In [ ]:
preds = [np.clip(mod.prediction(u,i),0,5) for u,i,_ in ratingsValid]
#preds = [np.clip((alpha+betaU[u] + betaI[i]),0,5) for u,i,_ in ratingsValid]
labels = [r for _,_,r in ratingsValid]
validMSE = MSE(preds, labels)#
validMSE

1.1134368375927364

## Latent Factor Model

In [ ]:
userIDs = {}
itemIDs = {}
interactions = []
for u,i,r in allRatings:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [ ]:
modelLFM = LatentFactorModel(globalAverage, 2,0.00001)

In [ ]:
def trainingStep(model, interactions):
    Nsamples = 30000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [ ]:
for i in range(100):
    obj = trainingStep(modelLFM, interactions[:190000])
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.820074
iteration 20, objective = 0.74256504
iteration 30, objective = 0.6948073
iteration 40, objective = 0.6760961
iteration 50, objective = 0.6409778
iteration 60, objective = 0.6210869
iteration 70, objective = 0.57874256
iteration 80, objective = 0.56941396
iteration 90, objective = 0.5695267
iteration 100, objective = 0.5642639


In [ ]:
preds = [np.clip((modelLFM.predict(userIDs[u], itemIDs[b]).numpy()),0,5) for u,b,_ in ratingsValid]
labels = [r for _,_,r in ratingsValid]
validMSE = MSE(preds, labels)
validMSE

1.5112906806164588

## Surprise

In [ ]:
!pip install surprise
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 7.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633967 sha256=f76c38c44e027bbcb8814854e1adc14b2480407a107b06298001e94346003c0d
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
reader = Reader(line_format='user item rating', sep=',', skip_lines = 1)
data = Dataset.load_from_file("train_Interactions.csv", reader=reader)
# param_grid = {'n_epochs': [47], 'lr_all': [0.0045, 0.00475, 0.005],
#               'reg_all': [0.5, 0.3, 0.4]}
# grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv = 3)   
# grid_search.fit(data)
# print(grid_search.best_params['rmse'])         


In [ ]:
trainset = data.build_full_trainset()

In [ ]:
model = SVD(n_epochs = 50, lr_all = 0.0047, reg_all = 0.32)

model.fit(trainset)
# predictions = model.test(testset)
# sse = 0
# for p in predictions:
#     sse += (p.r_ui - p.est)**2

# print(sse / len(predictions))

## Rating Answers

In [ ]:
suprise_pred = model.test(pairs_Ratings)
predictions = open("predictions_Rating.csv", 'w')
i = 0
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"):
      predictions.write(l)
      continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    #predictions.write(u + ',' + b + ',' + str(np.clip(mod.prediction(u,b),0,5)) + '\n')
    #predictions.write(u + ',' + b + ',' + str(np.clip((alpha+betaU[u] + betaI[b]),0,5)) + '\n')
    predictions.write(u + ',' + b + ',' + str(np.clip(suprise_pred[i].est,0,5)) + '\n')
    i+=1
    # try:
    #   pred = np.clip((modelLFM.predict(userIDs[u], itemIDs[b]).numpy()),0,5)
    # except:
    #   if u in userRatings:
    #     pred =  userAverage[u]
    #   else:
    #     pred = globalAverage
    #predictions.write(u + ',' + b + ',' + str(pred) + '\n')
  
predictions.close()

# Read Prediction

In [ ]:
#add 1 to every book which is read
ratingsValid = allRatings[190000:]
ratingsValid = [i + (1,) for i in ratingsValid]
ratingsValid[0]

('u59070515', 'b55084829', 5, 1)

In [ ]:
booksperUser = defaultdict(list)
userperBook = defaultdict(list)
for u,b,_ in allRatings:
    booksperUser[u].append(b)
    userperBook[b].append(u)

all_books = set(userperBook.keys())
booksperUserReadVal = defaultdict(list)
for u,b,_,_ in ratingsValid:
    booksperUserReadVal[u].append(b)

In [ ]:
#Adding random negative samples to the validation set
val_pairs = set()
for user in booksperUserReadVal.keys():
  no_books_read = len(booksperUserReadVal[user])
  books_not_read = list(all_books.difference(set(booksperUser[user])))
  for _ in range(no_books_read):
    book = random.choice(books_not_read)
    if (user,book, 'NA', 0) not in val_pairs:
      book = book
    else:
      while (user,book, 'NA', 0) in val_pairs:
        book = random.choice(books_not_read)
    val_pairs.add((user,book,'NA',0))
    ratingsValid.append((user,book,'NA',0)) 

len(ratingsValid)

20000

In [ ]:
# Copied from baseline code
bookCountAll = defaultdict(int)
bookCountTrain = defaultdict(int)
totalReadAll,totalReadTrain = 0,0

for user,book,_ in ratingsTrain:#readCSV("train_Interactions.csv.gz"):
    bookCountTrain[book] += 1
    totalReadTrain += 1
for user,book,_ in ratingsTrain:#readCSV("train_Interactions.csv.gz"):
    bookCountAll[book] += 1
    totalReadAll += 1

#mostPopular = [(bookCountTrain[x], x) for x in bookCountTrain]
mostPopular = [(bookCountAll[x], x) for x in bookCountAll]
mostPopular.sort()
mostPopular.reverse()
print(totalReadTrain)

avgBookCount = np.mean(list(bookCountAll.values()))
avgBookCount


190000


28.426092160383003

In [ ]:
# Changing threshold
return2 = set()
count = 0
threshold = 0.75
for bookCount, book in mostPopular:
    count += bookCount
    return2.add(book)
    if count > totalReadAll*threshold: break

In [ ]:
def Jaccard(s1, s2):
  if not isinstance(s1,set):
    s1 = set(s1)
  if not isinstance(s2,set):
    s2 = set(s2)
  numer = len(s1.intersection(s2))
  denom = len(s1.union(s2))
  if denom == 0:
      return 0
  return numer / denom

def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    if not isinstance(s1,set):
      s1 = set(s1)
    if not isinstance(s2,set):
      s2 = set(s2)
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

booksperUserTrain = defaultdict(list)
userperBookTrain = defaultdict(list)
for u,b,_ in ratingsTrain:
    booksperUserTrain[u].append(b)
    userperBookTrain[b].append(u)


In [ ]:
# def predictRating_41(user, book):
#   Csimilarities = []
#   Jsimilarities = []
#   for d in booksperUser[user]:
#     #d is book user has read in training
#     if d == book: continue
#     Jsimilarities.append(Jaccard(userperBook[book],userperBook[d]))
#     Csimilarities.append(CosineSet(userperBookTrain[book],userperBookTrain[d]))
#   if ((Csimilarities and max(Csimilarities) > 0.013) and (Jsimilarities and max(Jsimilarities) > 0.013) or \
#       len(userperBook[book])>40) and  book in return2:
#     return 1
#   return 0

def predictRating_41(user, book):
  Csimilarities = []
  Jsimilarities = []
  for d in booksperUser[user]:
    #d is book user has read in training
    if d == book: continue
    Jsimilarities.append(Jaccard(userperBook[book],userperBook[d]))
    Csimilarities.append(CosineSet(userperBook[book],userperBook[d]))
  mx = 0
  if Jsimilarities:
      mx = max(Jsimilarities)
  elif len(Jsimilarities) == 0:
    mx = 0
  if book in return2 or len(userperBook[book])>40:
    return 1 + mx
  return mx

# def predictRating_42(user, book):
#   Csimilarities = []
#   Jsimilarities = []
#   for d in userperBook[book]:
#     #d is user
#     if d == user: continue
#     Jsimilarities.append(Jaccard(booksperUserTrain[user],booksperUserTrain[d]))
#     Csimilarities.append(CosineSet(booksperUser[user],booksperUser[d]))
#   if ((Csimilarities and max(Csimilarities)) > 0.013 or \
#       len(userperBook[book]>40) or (Jsimilarities and max(Jsimilarities) >0.013)) and book in return2:
#     return 1
#   return 0

# preds = [predictRating_41(u,b) for u,b,_,_ in ratingsValid]
# correct = sum([ratingsValid[i][3]  == preds[i] for i in range(len(ratingsValid))])
# y = [x for _,_,_,x in ratingsValid]
# acc4 = correct/len(preds)
# acc4 #0.75955 #0.7619 

In [ ]:
# TP = sum([(p and l) for (p,l) in zip(preds, y)])
# FP = sum([(p and not l) for (p,l) in zip(preds, y)])
# TN = sum([(not p and not l) for (p,l) in zip(preds, y)])
# FN = sum([(not p and l) for (p,l) in zip(preds, y)])
# TP,FP, TN,FN

(7028, 1762, 8238, 2972)

In [ ]:
pairs_read_users = defaultdict(list)
for u,b in pairs_Read:
  pairs_read_users[u].append((predictRating_41(u,b),b))

final_scores = {}
for u in pairs_read_users:
  scores = sorted(pairs_read_users[u], reverse = True)
  for i in range(len(scores)):
    pred = 0
    b = scores[i][1]
    if i < len(scores)//2: #you are in first half
         pred = 1
    final_scores[(u,b)] =  pred
  

In [ ]:
predictions = open("predictions_Read.csv", 'w')
dct = {}
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    
    preds = final_scores[(u,b)]
    predictions.write(u + ',' + b + ","+str(preds)+"\n")

predictions.close()